We verify the recursive solution for calculating the following spectral matrix defined as follow:

S(omega) = (omega * I - G)^-1 * Y * (omega * I - G^T)^-1

In [1]:
import torch
import matplotlib.pyplot as plt
from spectre.spectrum_general import sim_solution
from spectre.spectrum_general import element_wise
from spectre.spectrum_general import recursive_solution_g
from spectre.model import HR
import numpy as np
import sympy as sp


plt.rc("text", usetex=True)
plt.rc("font", family="serif")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

fileloc = r"../figures/recursive_g/"

In [21]:
# Define the system size
n = 1

# Define the diagonal rate matrix Y
Y = torch.diag(torch.rand(n))

# Define the connectivity matrix G
G = torch.randn(n, n)
G = 0.9 * G / torch.max(torch.abs(torch.linalg.eigvals(G))) # redefining the spectral radius of G to be a 0.9

In [22]:
"""Define the frequency vectors"""
min_freq = 1e-4
max_freq = 10
n_freq_mat = 60
n_freq_rat = 1000

"""Define the frequency vectors for the matrix and rational solutions"""
freq_mat = torch.logspace(np.log10(min_freq), np.log10(max_freq), n_freq_mat)

In [23]:
# Find the matrix solution
omega = 1.0

S_matrix = torch.linalg.inv(omega * torch.eye(n) - G) @ Y @ torch.linalg.inv(omega * torch.eye(n) - G.T)


In [24]:
# Find the recursive solution
S_recursive = recursive_solution_g(G=G, L=torch.eye(n), S=Y)

# find the spectrum at omega

In [25]:
denm = sum([S_recursive.q[i] * omega ** i for i in range(2*n+1)])
num = sum([S_recursive.P[i] * omega ** i for i in range(2*n-1)], sp.zeros(n))

# numerical value of the spectrum
numerical_num = torch.from_numpy(np.array(num/denm, dtype=np.float32))

In [26]:
# compare S_matrix with numerical_num
print(torch.allclose(S_matrix, numerical_num, atol=1e-5))

False


In [27]:
S_matrix - numerical_num

tensor([[0.0667]])